# Output Preparation for the Nord_H2ub Spine Model

This jupyter notebook contains all routines for the preparation of the input data sources into a input data file for the model in Spine.

Authors: Johannes Giehl (jfg.eco@cbs.dk), Dana J. Hentschel (djh.eco@cbs.dk)

## General settings

### Packages:

In [1]:
import pandas as pd
import os
import openpyxl
import numpy as np

### define variables

In [2]:
excel_name = 'output_last_run'

### File paths:

In [13]:
#parent folder
parent_folder = '../02_basic_energy_model/.spinetoolbox/items/exporter/output'
folders = [f for f in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, f))]
if not folders:
    print("No folders found.")
else:
    latest_folder = max(folders, key=lambda x: os.path.getmtime(os.path.join(parent_folder, x)))
    latest_folder_path = os.path.join(parent_folder, latest_folder)
latest_folder_path = latest_folder_path.replace('\\', '/')
folder_path = latest_folder_path
folder_path += '/'

#prepared input data
output_file_path = '../02_output_data/01_basic_energy_model_outputs/'

In [14]:
#set name of the relevant files
#input file
output_old = 'Output_exported.xlsx'

## Workflow of the data preparation

### Data Import

In [21]:
df_output_raw = pd.read_excel(folder_path + output_old)
df_PV_prices = pd.read_excel('../01_input_data/02_input_prepared/Maersk_Example_Input_prepared.xlsx', sheet_name='Energy_prices')

### data frame preparation

In [105]:
#create a copy of the original output DataFrame
df_output = df_output_raw.copy()

# Replace NaN values with empty strings in the first three rows
df_output.iloc[:3] = df_output.iloc[:3].fillna('')

# Combine the old header with the strings from the first three rows for each column
new_headers = df_output.columns + '_' + df_output.iloc[0] + '_' + df_output.iloc[1] + '_' + df_output.iloc[2]

# Set the new headers
df_output.columns = new_headers

# Drop the first three rows
#df = df.drop([0, 1, 2])

# Reset the index
df_output.reset_index(drop=True, inplace=True)

# Rename the first column to "timeseries"
df_output.columns.values[0] = "timeseries"

### data adjustments

In [106]:
#calculate revenues from PV sales on the wholesale market
selected_column_name = None
for column_index in range(len(df_output.columns)):
    if df_output.iloc[0, column_index] == 'power_line_Wholesale_Kasso' \
        and df_output.iloc[1, column_index] == 'to_node' \
        and df_output.iloc[2, column_index] == 'Power_Wholesale':
        selected_column_name = df_output.columns[column_index]
        break

if selected_column_name:
    df_output['Revenue_from_PV'] = df_output[selected_column_name].iloc[3:] * df_PV_prices['Power_Wholesale_Out'].iloc[4]
else:
    print("Column with specified headers not found in output.")

In [107]:
#get total cost of the system
total_costs = df_output.filter(like='costs').iloc[3]
#get total revenue form PV power sale
total_PV_revenue = df_output['Revenue_from_PV'].sum()
#calculate cost without PV revenue
adjusted_costs = total_costs - total_PV_revenue

#create separate DataFrame for total and adjusted cost
df_system_cost_output = pd.DataFrame()
df_system_cost_output['Total_cost'] = total_costs
df_system_cost_output['PV_revenue'] = total_PV_revenue
df_system_cost_output['Total_adjusted_cost'] = adjusted_costs

In [111]:
# Identify columns to drop
columns_to_drop_1 = df_output.filter(like='costs').columns
# Drop the identified columns
df_output.drop(columns=columns_to_drop_1, inplace=True)

#columns_to_drop_2 = df_output.filter(like='unit_flow_op').columns
df_output.drop(columns=columns_to_drop_2, inplace=True)

### Creating one combined excel and export

In [8]:
with pd.ExcelWriter(output_file_path + excel_name + '.xlsx') as writer:
    df_output.to_excel(writer, sheet_name='Total costs')

In [112]:
df_output.iloc[:,[15,16,17]]

,node_state_E-Methanol_storage_Kasso__realisation,node_state.1_Hydrogen_storage_Kasso__realisation,unit_flow_CO2_Vaporizer_from_node_Carbon_Dioxide
0,E-Methanol_storage_Kasso,Hydrogen_storage_Kasso,CO2_Vaporizer
1,,,from_node
2,realisation,realisation,Carbon_Dioxide
3,0,0,7.167793
4,4.33,0,1.241462
5,0,11.916667,5.926331
6,28.8596,0,11.141503
7,26.6244,0,8.533917
8,36.3892,0,8.533917
9,34.4056,14.083333,5.452225
